##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.0094389 , -0.12249541,  0.8785145 , -0.56476504,  0.14063849,
        -0.6773041 ,  0.3953506 ,  0.33076695,  0.868703  , -1.237788  ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.10758542, 0.0844648 , 0.06330214, 0.08534131, 0.08827639,
        0.19123103, 0.07983079, 0.14475977, 0.1053158 , 0.04989255]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [5]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [6]:
loss_fn(y_train[:1], predictions).numpy()

3.1672733

In [7]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [10]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 56us/sample - loss: 0.2977 - accuracy: 0.9132
Epoch 2/5
60000/60000 [==============================] - 3s 52us/sample - loss: 0.1426 - accuracy: 0.9567
Epoch 3/5
60000/60000 [==============================] - 3s 53us/sample - loss: 0.1080 - accuracy: 0.9675
Epoch 4/5
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0864 - accuracy: 0.9731
Epoch 5/5
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0728 - accuracy: 0.9768


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 1s - loss: 0.0747 - accuracy: 0.9767


[0.07468590899039991, 0.9767]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.23901875e-09, 9.67688818e-10, 1.28809791e-06, 7.36907677e-05,
        2.41914800e-12, 2.77970275e-06, 1.12970383e-13, 9.99920487e-01,
        1.04393408e-07, 1.64716926e-06],
       [1.86067297e-07, 2.38574994e-05, 9.99640226e-01, 1.99171627e-04,
        7.00788135e-15, 4.02210799e-06, 3.78847091e-07, 2.81901526e-14,
        1.32185771e-04, 5.54750690e-12],
       [2.75134255e-07, 9.98852372e-01, 1.01070058e-04, 9.15505825e-06,
        1.97388454e-05, 1.32591385e-05, 1.05952176e-05, 6.89086737e-04,
        3.04044544e-04, 4.13105369e-07],
       [9.99725759e-01, 2.06162620e-09, 2.10127924e-04, 7.02684346e-08,
        4.45656451e-06, 1.28747013e-06, 3.78554869e-05, 1.83260127e-05,
        1.58706598e-07, 1.82478050e-06],
       [4.65019923e-07, 1.98255279e-09, 1.35111679e-06, 8.00903877e-08,
        9.99255240e-01, 1.27085741e-07, 1.47501922e-07, 1.17184109e-05,
        2.76722290e-07, 7.30475818e-04]], dtype=float32)>